<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Ejercicios creados a partir de 2019-2 por Equipo Docente IIC2233. </font>
<font size='1'> Actualizados en 2023-2.</font>
</p>


# Ejercicios propuestos: Interfaces gráficas I
## Diseño *front-end - back-end*

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de interfaces gráficas I. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

### Ejercicio 1: Ventana para calculadora

A partir del siguiente código entregado, debes utilizarlo como el *back-end* de un programa, y debes crear su *front-end*. Este *back-end* modela una calculadora, capaz de operar dos valores según suma, resta, multiplicación y división, además de revisar el *input* previamente.

Este *back-end* incluye una señal llamada `senal_mostrar_resultado` que se emitirá despues de procesar cualquier *input* que le llegue al método `validar_input`.

```python
# calculadora.py
from PyQt5.QtCore import pyqtSignal, QObject


class Calculadora(QObject):

    senal_mostrar_resultado = pyqtSignal(str)

    def suma(self, valor1, valor2):
        string_resultado = str(int(valor1) + int(valor2))
        self.senal_mostrar_resultado.emit(string_resultado)

    def resta(self, valor1, valor2):
        string_resultado = str(int(valor1) - int(valor2))
        self.senal_mostrar_resultado.emit(string_resultado)

    def cuociente(self, valor1, valor2):
        string_resultado = str(float(valor1) / float(valor2))
        self.senal_mostrar_resultado.emit(string_resultado)

    def multiplicacion(self, valor1, valor2):
        string_resultado = str(int(valor1) * int(valor2))
        self.senal_mostrar_resultado.emit(string_resultado)

    def validar_input(self, accion):
        # método que recibe como señal un diccionario de la forma
        # accion = {'operación': operacion, 'valor1': valor1, 'valor2: valor2'}
        if accion['valor1'].isnumeric() and accion['valor2'].isnumeric():
            if accion['operacion'] == "sumar":
                self.suma(accion['valor1'], accion['valor2'])
            elif accion['operacion'] == "restar":
                self.resta(accion['valor1'], accion['valor2'])
            elif accion['operacion'] == "multiplicar":
                self.multiplicacion(accion['valor1'], accion['valor2'])
            elif accion['operacion'] == "dividir":
                if accion['valor2'] == 0:
                    self.senal_mostrar_resultado.emit('Error: No dividir por cero')
                else:
                    self.cuociente(accion['valor1'], accion['valor2'])
        else:
            self.senal_mostrar_resultado.emit('Error: Input inválido')
```

El código de *back-end* se espera lo coloques en un módulo `calculadora.py`, mientras que el *front-end* y código principal utilicen dicho módulo. 

Para el *front-end* y código principal se te provee una base a continuación. Completa la clase `Ventana` para crear una ventana capaz de recibir dos valores, con botones para realizar las operaciones sumar, restar, multiplicar y dividir, y capaz de mostrar el resultado de operación. Si algún valor ingresado no es un número, muestra el mensaje: `'Error: Input inválido'`.
Recuerda conectar tu *front-end* y *back-end* a través de señales en el código principal (`if __name__ == '__main__'`).

```python
# ventana_ejercicio_1.py
import sys
from PyQt6.QtCore import pyqtSignal, QObject
from PyQt6.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QLineEdit)
from calculadora import Calculadora


class Ventana(QWidget):
    senal_calcular = pyqtSignal(dict)
    def __init__(self):
        super().__init__()
        # completar


if __name__ == '__main__':
    def hook(type, value, traceback):
        print(type)
        print(traceback)

    sys.__excepthook__ = hook
    
    app = QApplication([])
    calculadora = Calculadora()
    ventana = Ventana()

    # conectar señales a continuación
    # Recuerda conectar cada señal con el método de la clase que corresponda

    
    sys.exit(app.exec())
```

A modo de ejemplo, puedes seguir la siguiente idea para la ventana:

![](img/prob-4-1.png)

### Ejercicio 2: Separa el programa

En este ejercicio, te entregamos un programa listo e implementado, pero lamentablemente, su desarrollador no siguió la idea de separación entre *back-end* y *front-end*. Debes tomar el programa y modificarlo de tal forma de generar una separación entre lógica de programa (*back-end*) y lógica de interfaz gráfica (*front-end*).

El programa muestra en pantalla una imagen de una pitón, que comienza al centro de una grilla invisible de tamaño 3 por 3. Al usar las teclas `WASD`, se puede mover la pitón a través de la grilla según la dirección de la tecla. El programa se encarga de cargar las imágenes en pantalla, conectar la interacción de teclas, realizar el movimiento de la pitón, y actualizar la grilla luego de cambiar de posición. Crea un módulo separado a la ventana, que representará al *back-end*, de tal forma que este se encargue de la posición interna de la pitón, y de actualizar dicha posición cuando recibe una dirección de movimiento. En el código de la ventana solo puede quedar código relacionado al cargado y actualización de celdas, interacción mediante teclas, además de las conexiones con el *back-end*. Utiliza señales para conectar el nuevo módulo de *back-end* con la clase `Ventana` del *front-end*.

***Idea:* La interacción mediante señales puede ser de la siguiente forma: una señal se dedica a enviar desde el *front-end* al *back-end* la dirección de movimiento después de apretar una tecla, y otra señal va en sentido contrario enviando la posición antigua y nueva de la pitón, para que se actualice en la ventana.**

Hay muchas soluciones distintas para esto. Intenta realizar una al menos, y luego puedes cuestionarte como mejorarlo.

```python
import sys
from os import path
from PyQt6.QtCore import pyqtSignal, QObject, Qt
from PyQt6.QtWidgets import (QApplication, QWidget, QLabel)
from PyQt6.QtGui import QPixmap


class Ventana(QWidget):
    def __init__(self):
        super().__init__()
        self.posicion = (1, 1)
        self.ruta_imagen = path.join('img', 'python_icon.png')
        self.inicializa_gui()

    def inicializa_gui(self):
        self.setGeometry(300, 300, 300, 300)
        self.etiquetas = {}
        for fila in range(3):
            for columna in range(3):
                nueva_etiqueta = QLabel(self)
                nueva_etiqueta.setGeometry(fila * 100, columna * 100, 100, 100)
                if (fila, columna) == self.posicion:
                    pixmap = QPixmap(self.ruta_imagen)
                else:
                    pixmap = QPixmap(100, 100)
                    pixmap.fill(Qt.white)
                nueva_etiqueta.setPixmap(pixmap)
                nueva_etiqueta.setScaledContents(True)
                self.etiquetas[(fila, columna)] = nueva_etiqueta

    def mover_python(self, direccion):
        ex_posicion = self.posicion
        if direccion == 'arriba':
            self.posicion = (ex_posicion[0], (ex_posicion[1] - 1) % 3)
        elif direccion == 'abajo':
            self.posicion = (ex_posicion[0], (ex_posicion[1] + 1) % 3)
        elif direccion == 'izquierda':
            self.posicion = ((ex_posicion[0] - 1) % 3, ex_posicion[1])
        elif direccion == 'derecha':
            self.posicion = ((ex_posicion[0] + 1) % 3, ex_posicion[1])
        if ex_posicion != self.posicion:
            pixmap = QPixmap(100, 100)
            pixmap.fill(Qt.white)
            self.etiquetas[ex_posicion].setPixmap(pixmap)
            pixmap = QPixmap(self.ruta_imagen)
            self.etiquetas[self.posicion].setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == 65:  # A
            self.mover_python('izquierda')
        elif event.key() == 87:  # W
            self.mover_python('arriba')
        elif event.key() == 83:  # S
            self.mover_python('abajo')
        elif event.key() == 68:  # D
            self.mover_python('derecha')


if __name__ == '__main__':
    def hook(type, value, traceback):
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana = Ventana()
    ventana.show()
    sys.exit(app.exec())
```